In [0]:
spark

In [0]:
dbutils.fs.mount(source='wasbs://landing@week17storage.blob.core.windows.net',mount_point='/mnt/week11assignment/landing',
extra_configs={'fs.azure.account.key.week17storage.blob.core.windows.net':'DwezHFxwgpDba4/fh4P6oR3TlTZYUoJTggu4PIGhSt5AF23lMRBi9yfG6tfOtR666VhUlFexopS2+AStge7HnQ=='})

True

In [0]:
dbutils.fs.mount(source='wasbs://reporting@week17storage.blob.core.windows.net',mount_point='/mnt/week11assignment/reporting',
extra_configs={'fs.azure.account.key.week17storage.blob.core.windows.net':'DwezHFxwgpDba4/fh4P6oR3TlTZYUoJTggu4PIGhSt5AF23lMRBi9yfG6tfOtR666VhUlFexopS2+AStge7HnQ=='})

True

In [0]:
dbutils.fs.mount(source='wasbs://staging@week17storage.blob.core.windows.net',mount_point='/mnt/week11assignment/staging',
extra_configs={'fs.azure.account.key.week17storage.blob.core.windows.net':'DwezHFxwgpDba4/fh4P6oR3TlTZYUoJTggu4PIGhSt5AF23lMRBi9yfG6tfOtR666VhUlFexopS2+AStge7HnQ=='})

True

In [0]:
order_schema='order_id String,order_date date,order_customer_id String,order_status String'

In [0]:
print(order_schema)

order_id String,order_date date,order_customer_id String,order_status String


In [0]:
orders_df=spark.read \
    .format("csv") \
    .schema(order_schema) \
    .load("dbfs:/mnt/week11assignment/landing/orders_data.csv")

In [0]:
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)



In [0]:
orders_df.show(5)

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|       1|2013-07-25|            11599|         CLOSED|
|       2|2013-07-25|              256|PENDING_PAYMENT|
|       3|2013-07-25|            12111|       COMPLETE|
|       4|2013-07-25|             8827|         CLOSED|
|       5|2013-07-25|            11318|       COMPLETE|
+--------+----------+-----------------+---------------+
only showing top 5 rows



In [0]:
customer_schema='customer_id string,customer_fname string,customer_lname string,customer_email string,customer_password string,customer_street string,customer_city string,customer_state string,customer_zipcode string'

In [0]:
print(customer_schema)

customer_id string,customer_fname string,customer_lname string,customer_email string,customer_password string,customer_street string,customer_city string,customer_state string,customer_zipcode string


In [0]:
customer_df=spark.read \
    .format("csv") \
    .schema(customer_schema) \
    .load('dbfs:/mnt/week11assignment/landing/customers_data.csv')

In [0]:
customer_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)



In [0]:
from pyspark.sql.functions import *

In [0]:
orders_new_df=orders_df.withColumn("order_year",year("order_date")).withColumn("order_month",month("order_date"))

In [0]:
orders_new_df.show(5)

+--------+----------+-----------------+---------------+----------+-----------+
|order_id|order_date|order_customer_id|   order_status|order_year|order_month|
+--------+----------+-----------------+---------------+----------+-----------+
|       1|2013-07-25|            11599|         CLOSED|      2013|          7|
|       2|2013-07-25|              256|PENDING_PAYMENT|      2013|          7|
|       3|2013-07-25|            12111|       COMPLETE|      2013|          7|
|       4|2013-07-25|             8827|         CLOSED|      2013|          7|
|       5|2013-07-25|            11318|       COMPLETE|      2013|          7|
+--------+----------+-----------------+---------------+----------+-----------+
only showing top 5 rows



In [0]:
orders_new_df.write \
. format("parquet") \
. mode("overwrite") \
.partitionBy("order_year","order_status") \
.save("/mnt/week11assignment/staging/orders")

In [0]:
customer_df.write \
.format("parquet") \
.mode("overwrite") \
.partitionBy("customer_state") \
.save("/mnt/week11assignment/staging/customers")

In [0]:
customer_staging_df=spark.read \
    .format("parquet") \
    .schema(customer_schema) \
    .load('dbfs:/mnt/week11assignment/staging/customers')

In [0]:
orders_staging_df=spark.read \
    .format("parquet") \
    .option("header",True) \
    .load("dbfs:/mnt/week11assignment/staging/orders")

In [0]:
customer_staging_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_fname: string (nullable = true)
 |-- customer_lname: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_password: string (nullable = true)
 |-- customer_street: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_zipcode: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [0]:
orders_staging_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_month: integer (nullable = true)
 |-- order_year: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [0]:
joined_df=orders_staging_df.join(broadcast(customer_staging_df),expr("order_customer_id=customer_id"))

In [0]:
joined_df.show(5)

+--------+----------+-----------------+-----------+----------+------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+----------------+--------------+
|order_id|order_date|order_customer_id|order_month|order_year|order_status|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_zipcode|customer_state|
+--------+----------+-----------------+-----------+----------+------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+----------------+--------------+
|   68883|2014-07-23|             5533|          7|      2014|    COMPLETE|       5533|          Mary|      Harrison|     XXXXXXXXX|        XXXXXXXXX|852 Rocky Brook M...|       Caguas|           00725|            PR|
|   68880|2014-07-13|             1117|          7|      2014|    COMPLETE|       1117|        Arthur|        Maddox|     XXXXXX

In [0]:
report_df=joined_df.select("order_id","order_date","order_customer_id","order_month","order_year","order_status","customer_fname","customer_lname","customer_city","customer_zipcode","customer_state")

In [0]:
report_df.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy("order_year","customer_state","customer_city","order_status") \
    .save("/mnt/week11assignment/reporting")

In [0]:
report_df.createOrReplaceTempView("order_customer")

In [0]:
report_new_df=spark.read \
    .format("parquet") \
    .option("header","true") \
    .load("/mnt/week11assignment/reporting")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3178365256526096>, line 4
      1 report_new_df=spark.read \
      2     .format("parquet") \
      3     .option("header","true") \
----> 4     .load("/mnt/week11assignment/reporting")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:307, in DataFrameReader.load(self, path, format, schema, **options)
    305 self.options(**options)
    306 if isinstance(path, str):
--> 307     return self._df(self._jreader.load(path))
    308 elif path is not None:

In [0]:
report_df.select(expr("order_customer_id").alias("customer_id"),expr("concat(customer_fname,'',customer_lname)").alias("customer_name"),"order_date","customer_city","customer_state","order_status","order_year","order_month").createOrReplaceTempView("order_cust_vw")

In [0]:
spark.sql("select * from order_cust_vw limit 10").show()

+-----------+-------------+----------+-------------+--------------+------------+----------+-----------+
|customer_id|customer_name|order_date|customer_city|customer_state|order_status|order_year|order_month|
+-----------+-------------+----------+-------------+--------------+------------+----------+-----------+
|       5533| MaryHarrison|2014-07-23|       Caguas|            PR|    COMPLETE|      2014|          7|
|       1117| ArthurMaddox|2014-07-13|       Caguas|            PR|    COMPLETE|      2014|          7|
|        778|  JohnBuckley|2014-07-09|       Caguas|            PR|    COMPLETE|      2014|          7|
|       6753| JudithHester|2014-07-08|  Winter Park|            FL|    COMPLETE|      2014|          7|
|       4124|  JerryLarsen|2014-07-06|       Caguas|            PR|    COMPLETE|      2014|          7|
|       1601|  MarkPadilla|2014-07-03|       Caguas|            PR|    COMPLETE|      2014|          7|
|       3354|RussellFlores|2014-06-29|       Caguas|            

In [0]:
spark.sql("cache table order_cust_vw")

DataFrame[]

In [0]:
%sql
select * from order_cust_vw where customer_state='TX' AND order_status='COMPLETE' AND order_year='2014';


customer_id,customer_name,order_date,customer_city,customer_state,order_status,order_year,order_month
312,JackSmith,2014-06-04,Houston,TX,COMPLETE,2014,6
8451,JaniceNovak,2014-05-25,Houston,TX,COMPLETE,2014,5
2564,PhillipSmith,2014-04-05,Amarillo,TX,COMPLETE,2014,4
2163,NicholasWright,2014-07-10,Irving,TX,COMPLETE,2014,7
5259,DennisSmith,2014-06-27,Houston,TX,COMPLETE,2014,6
10433,ElizabethBall,2014-06-26,Houston,TX,COMPLETE,2014,6
8952,JosephBrooks,2014-06-11,Mission,TX,COMPLETE,2014,6
2202,MarySmith,2014-06-05,Mesquite,TX,COMPLETE,2014,6
9263,JoeSmith,2014-04-28,Dallas,TX,COMPLETE,2014,4
8391,RobertSmith,2014-04-21,Amarillo,TX,COMPLETE,2014,4


In [0]:
dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [0]:
dbutils.widgets.dropdown(name='state_input',defaultValue='TX',choices=['TX','NY','CA'],label="State:")

In [0]:
dbutils.widgets.dropdown(name='status_input',defaultValue='COMPLETE',choices=['COMPLETE','PENDING_PAYMENT','CLOSED','PROCESSING'],label="Status:")

In [0]:
dbutils.widgets.text(name='year_input',defaultValue="2014",label="Year:")

In [0]:
state=dbutils.widgets.get('state_input')
status=dbutils.widgets.get('status_input')
year=dbutils.widgets.get('year_input')

In [0]:
display(spark.sql(f"SELECT distinct customer_name FROM order_cust_vw WHERE customer_state='{state}'AND order_status='{status}' AND order_year={year}"))


customer_name
MichelleRose
StephenLee
KimberlySweeney
JohnFuentes
BenjaminSmith
RebeccaWatkins
LoriSmith
MaryJohnson
MargaretVilla
AlbertSmith


In [0]:
dbutils.widgets.dropdown("month_input","1",["1","2","3","4","5","6","7","8","9","10","11","12"],"Month:")

In [0]:
month=dbutils.widgets.get("month_input")


In [0]:
spark.sql(f"SELECT count(customer_id)FROM order_cust_vw WHERE customer_state='{state}' AND order_status='{status}'AND order_year={year} AND order_month={month}")

DataFrame[count(customer_id): bigint]